### Batch Runner of ModularCirc

This file samples the input parameters of the Korakianitis model then batch solves this using ModularCirc. The raw output, pressure traces, cardiac output are all saved. Additionally a PCA is run on the pressure traces and are also saved.

In [1]:
from ModularCirc.Models.KorakianitisMixedModel import KorakianitisMixedModel, KorakianitisMixedModel_parameters, TEMPLATE_TIME_SETUP_DICT
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
from ModularCirc import BatchRunner

In [3]:
br = BatchRunner('Sobol', 0) 
#'LHS' : LatinHypercube,
#'Sobol' : Sobol,
#'Halton': Halton,

In [ ]:
path = os.getcwd()
path

In [5]:
# Parameters_01 = Korakianitis Model
br.setup_sampler('parameters_pulmonary.json')

In [ ]:
# Set number of samples 
n_sample = 2000
br.sample(n_sample)

In [ ]:
br.samples

In [ ]:
TEMPLATE_TIME_SETUP_DICT

In [9]:
map_ = {
    'delay' : ['la.delay', 'ra.delay'],
    'td0'   : ['lv.td0',   'rv.td0' ],
    'tr'    : ['lv.tr',    'rv.tr'  ],
    'tpww'  : ['la.tpww',  'ra.tpww'],
}
br.map_sample_timings(
    ref_time=1.,
    map=map_
    )

In [ ]:
br.samples.columns

In [11]:
br._samples[['lv.td', 'rv.td']] = br._samples[['lv.tr', 'rv.tr']].values + br._samples[['lv.td0', 'rv.td0']].values
br._samples.drop(['lv.td0', 'rv.td0'], axis=1, inplace=True)

In [ ]:
br.samples

In [ ]:
br.map_vessel_volume()


In [ ]:
# Quick stats on some parameters
br._samples[['svn.c', 'pat.r', 'pat.c', 'svn.c']].describe().T

In [16]:
br.setup_model(model=KorakianitisMixedModel, po=KorakianitisMixedModel_parameters, time_setup=TEMPLATE_TIME_SETUP_DICT)

In [ ]:
input_header = ','.join(br.samples.columns)
input_header

In [18]:
# Save sampled inputs to CSV
os.system(f'mkdir -p {path+'/Input'}')
np.savetxt(path+f'/Input/input_{n_sample}.csv', br.samples, header=input_header, delimiter=',')

In [ ]:
n_sample

In [ ]:
os.system(f'mkdir -p {path+'/Outputs/Output_2000/Output_2000_raw'}')
test = br.run_batch(n_jobs=5, output_path=path+f'/Outputs/Output_{n_sample}/Output_{n_sample}_raw')

In [ ]:
test

In [ ]:
# Summary stats of output of first realisation
ind = 0
test[ind].loc[ind].describe()

### It looks like there is a bug in the code and some entries of test are boolean - Maybe it did not converge here? 

In [ ]:
# Check for bool values in the list
bool_indices = [index for index, value in enumerate(test) if isinstance(value, bool)]

if bool_indices:
    print(f"Boolean values found at indices: {bool_indices}")
else:
    print("No boolean values found in the list.")

### From now on I will only use items in test which are not boolean. 

In [ ]:
# Initialize the plot
fig, ax = plt.subplots()

# Loop over all realizations
for ind in range(len(test)): 
    if not isinstance(test[ind], bool):
        # Adjust time and pressure trace for each realization
        t = test[ind].loc[ind]['T'] - test[ind].loc[ind]['T'].loc[0]  # Time adjustment
        p_pat = test[ind].loc[ind]['p_pat']  # Pressure transient

        # Plot the pressure transient for each realization
        ax.plot(t, p_pat, label=f'Realisation {ind}')

# Set labels and title
ax.set_xlabel('Time (seconds)')
ax.set_ylabel('Pressure (mmHg)')
ax.set_title('Pressure Transients in Arterial Tree')

# Add legend to the plot
# ax.legend()

# Display the plot
plt.show()

In [ ]:
ind = 0
p_pat_raw = test[ind].loc[ind]['p_pat']
p_pat_raw



In [ ]:
## Create directory for pessure traces 
os.system(f'mkdir -p {path}/Outputs/Output_2000/pressure_traces')


## Save individual pressure traces, CO and dt

In [25]:
# Create column headers
headers = list(range(100)) + ['CO', 'dt', 'EF']

# List to collect all pressure traces
pressure_traces_list = []

for ind in range(len(test)):
    if not isinstance(test[ind], bool):
     p_pat_raw = test[ind].loc[ind]['p_pat'].values.copy()
     T = test[ind].loc[ind]['T'].values.copy()
     T_resample = np.linspace(T[0], T[-1], 100)

     # Interpolate pressure for 100 timesteps from 1000
     p_pat_resampled = np.interp(T_resample, T, p_pat_raw)

     q_pat = test[ind].loc[ind]['q_pat'].values.copy()
     CO = np.sum(q_pat) * (T[1] - T[0]) / (T[-1] - T[0]) * 60. / 1000.  # L / min
     v_rv = test[ind].loc[ind]['v_rv'].values.copy()
     EF = (np.max(v_rv) - np.min(v_rv)) / np.max(v_rv)
    
     # Record time interval, approx T (input param) / 100, there are some rounding differences due to interpolation
     tl = T_resample - test[ind].loc[ind]['T'].iloc[0]
     dt = np.diff(tl)[0]

     # Create a 2D array for saving
     pressure_trace = np.hstack((p_pat_resampled, [CO], [dt], [EF]))
     pressure_traces_list.append(pressure_trace)

     # Save individual pressure trace to CSV with headers
     individual_df = pd.DataFrame([pressure_trace], columns=headers)
     individual_df.to_csv(f'{path}/Outputs/Output_{n_sample}/pressure_traces/pressuretrace_{ind}.csv', index=False)

# Convert the list of pressure traces to a DataFrame
pressure_traces_df = pd.DataFrame(pressure_traces_list, columns=headers)

# Save the DataFrame to a single CSV file with headers
pressure_traces_df.to_csv(f'{path}/Outputs/Output_{n_sample}/pressure_traces/all_pressure_traces.csv', index=False)

In [ ]:
# Initialize the plot
fig, ax = plt.subplots()

# Loop over all realizations
for ind in range(len(test)): 
    if not isinstance(test[ind], bool):
     p_pat_raw = test[ind].loc[ind]['p_pat'].values.copy()
     T = test[ind].loc[ind]['T'] - test[ind].loc[ind]['T'].loc[0]  # Time adjustment
     T = T.values.copy()
     T_resample = np.linspace(T[0], T[-1], 100)
    

     # Interpolate pressure for 100 timesteps from 1000
     p_pat_resampled = np.interp(T_resample, T, p_pat_raw)

     # Plot the interpolated pressure transient for each realization
     ax.plot(list(range(100)), p_pat_resampled, label=f'Realisation {ind}')

# Set labels and title
ax.set_xlabel('Time index')
ax.set_ylabel('Pressure (mmHg)')
ax.set_title('Pressure Transients in Arterial Tree')

# Add legend to the plot
#ax.legend()

# Display the plot
plt.show()

### Conducting PCA on Pressure Traces

In [ ]:
path

In [ ]:
# Import Data

# Define the path to the folder containing the CSV files
folder_path = f'{path}/Outputs/Output_{n_sample}/pressure_traces'

df = pd.read_csv(f'{folder_path}/all_pressure_traces.csv')

# Print the DataFrame
print(df)

In [ ]:
import sklearn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

# Copy the data and separate the target variable (only pressure traces)
#X = df # traces + CO + dt
X = df.iloc[:,:101].copy() # traces + CO
#X = df.iloc[:,:100].copy() # traces only

# Create an instance of StandardScaler
scaler = StandardScaler()

# Fit the scaler to the data and transform it - standardize
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_scaled)

# Convert to dataframe
component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
X_pca = pd.DataFrame(X_pca, columns=component_names, index=df.index)

X_pca.head()



In [ ]:
# Plot Histograms
X_pca.hist(bins=30, figsize=(15, 13), layout=(5, 2), alpha=0.7, color='orange')
plt.suptitle('Histograms of the First 10 Principal Components')
plt.show()

In [44]:
def plot_variance(pca, width=8, dpi=100):
    # Create figure
    fig, axs = plt.subplots(1, 2)
    n = pca.n_components_
    grid = np.arange(1, n + 1)
    # Explained variance
    explained_variance_ratio = pca.explained_variance_ratio_
    axs[0].bar(grid, explained_variance_ratio, log=True)
    axs[0].set(
        xlabel="Component", title="% Explained Variance", ylim=(0.0, 1.0)
    )

    # Cumulative Variance
    cumulative_explained_variance = np.cumsum(explained_variance_ratio)
    axs[1].semilogy(grid, cumulative_explained_variance, "o-")
    axs[1].set(
        xlabel="Component", title="% Cumulative Variance", 
    )
    # Set up figure
    fig.set(figwidth=8, dpi=100)
    fig.tight_layout()
    return axs

In [ ]:
plot_variance(pca)

In [ ]:
path

In [47]:
os.system(f'mkdir -p {path}/Outputs/Output_{n_sample}/PCA')

# Save first 3 Principle Component data
for i in list(range(3)):

 PC = X_pca.iloc[:,i]
 PC.to_csv(f'{path}/Outputs/Output_{n_sample}/PCA/CO_PC{i+1}.csv', index=False)






### Re-import all data to write as single xlsx file

In [60]:
dataframes = {}

# Read PCA data
for i in range(3):
    df_name = f'y_PC{i+1}'  # Create the dataframe name
    dataframes[df_name] = pd.read_csv(f'{path}/Outputs/Output_{n_sample}/PCA/PC{i+1}.csv')  # Read and store the dataframe
    dataframes[f'all_{df_name}'] =  pd.read_csv(f'{path}/Outputs/Output_{n_sample}/PCA/all_PC{i+1}.csv') 
    dataframes[f'CO_{df_name}'] =  pd.read_csv(f'{path}/Outputs/Output_{n_sample}/PCA/CO_PC{i+1}.csv') 

y_PC1 = dataframes['y_PC1']
y_PC2 = dataframes['y_PC2']
y_PC3 = dataframes['y_PC3']

all_y_PC1 = dataframes['all_y_PC1']
all_y_PC2 = dataframes['all_y_PC2']
all_y_PC3 = dataframes['all_y_PC3']

CO_y_PC1 = dataframes['CO_y_PC1']
CO_y_PC2 = dataframes['CO_y_PC2']
CO_y_PC3 = dataframes['CO_y_PC3']

df_pressure = pd.read_csv(f'{path}/Outputs/Output_{n_sample}/pressure_traces/all_pressure_traces.csv')
cardiac_output = df_pressure.iloc[:,100:101]
ejection_frac = df_pressure.iloc[:,102:103]

mean_press = df_pressure.iloc[:,:100].mean(axis=1).to_frame(name='mean_press')
max_press = df_pressure.iloc[:,:100].max(axis=1).to_frame(name='max_press')
min_press = df_pressure.iloc[:,:100].min(axis=1).to_frame(name='min_press')


### Write all data to s single combined excel file


In [ ]:
import pandas as pd
from pandas import ExcelWriter

# Define output file path
output_file = f'{path}/Outputs/Output_{n_sample}/combined_all_outputdata_{n_sample}.xlsx'

# Use ExcelWriter to write all dataframes into separate sheets
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    # Write PCA data
    for i in range(3):
        dataframes[f'y_PC{i+1}'].to_excel(writer, sheet_name=f'y_PC{i+1}', index=False)
        dataframes[f'all_y_PC{i+1}'].to_excel(writer, sheet_name=f'all_y_PC{i+1}', index=False)
        dataframes[f'CO_y_PC{i+1}'].to_excel(writer, sheet_name=f'CO_y_PC{i+1}', index=False)
    
    # Write pressure and cardiac output data
    df_pressure.to_excel(writer, sheet_name='all_pressure_traces', index=False)
    cardiac_output.to_excel(writer, sheet_name='cardiac_output', index=False)
    mean_press.to_excel(writer, sheet_name='mean_pressure', index=False)
    max_press.to_excel(writer, sheet_name='max_pressure', index=False)
    min_press.to_excel(writer, sheet_name='min_pressure', index=False)
    ejection_frac.to_excel(writer, sheet_name='EF', index=False)

print(f"Combined data saved to {output_file}")